In [1]:
import numpy as np
import pandas as pd
from surprise import Dataset, Reader, SVD, KNNBasic
from surprise.model_selection import GridSearchCV, cross_validate
from surprise.dump import dump, load

# 1. Завантаження датасету
from surprise.dataset import DatasetAutoFolds
data = DatasetAutoFolds(ratings_file='ml-100k/u.data', 
                        reader=Reader(line_format='user item rating timestamp', sep='\t'))



ModuleNotFoundError: No module named 'surprise'

In [ ]:
# 2. Виведення перших 5 рядків
df = pd.DataFrame(data.raw_ratings, columns=['user', 'item', 'rating', 'timestamp'])
print("Перші 5 рядків датасету:")
print(df.head())



In [ ]:
# 3. Реалізація двох алгоритмів
svd = SVD()
knn = KNNBasic()



In [ ]:
# 4. Крос-валідація та підбір параметрів
param_grid_svd = {'n_epochs': [5, 10, 20], 'lr_all': [0.002, 0.005], 'reg_all': [0.4, 0.6]}
gs_svd = GridSearchCV(SVD, param_grid_svd, measures=['mae'], cv=3)
gs_svd.fit(data.build_full_trainset())

param_grid_knn = {'k': [20, 40, 60], 'sim_options': {'name': ['msd', 'cosine'], 'user_based': [False, True]}}
gs_knn = GridSearchCV(KNNBasic, param_grid_knn, measures=['mae'], cv=3)
gs_knn.fit(data.build_full_trainset())

# Вибір найкращих параметрів
best_svd = gs_svd.best_estimator['mae']
best_knn = gs_knn.best_estimator['mae']

print("Найкращі параметри для SVD:", gs_svd.best_params['mae'])
print("Найкращі параметри для KNN:", gs_knn.best_params['mae'])


In [ ]:

# 5. Вибір найкращого алгоритму на основі MAE
results_svd = cross_validate(best_svd, data, measures=['MAE'], cv=5, verbose=False)
results_knn = cross_validate(best_knn, data, measures=['MAE'], cv=5, verbose=False)

mae_svd = np.mean(results_svd['test_mae'])
mae_knn = np.mean(results_knn['test_mae'])

print(f"MAE для SVD: {mae_svd}")
print(f"MAE для KNN: {mae_knn}")

best_algo = best_svd if mae_svd < mae_knn else best_knn
print(f"Найкращий алгоритм: {'SVD' if mae_svd < mae_knn else 'KNN'}")

# Тренування найкращої моделі на всьому датасеті
trainset = data.build_full_trainset()
best_algo.fit(trainset)



In [ ]:
# 6. Рекомендації для конкретного користувача
user_id = 1  # Можна змінити на будь-якого іншого користувача
movie_ids = range(1, 1683)  # Всі фільми в датасеті

# Отримання прогнозів для всіх фільмів для вказаного користувача
predictions = [best_algo.predict(user_id, item_id) for item_id in movie_ids]

# Сортування прогнозів за оцінкою
top_predictions = sorted(predictions, key=lambda x: x.est, reverse=True)[:10]

print(f"\nТоп-10 рекомендацій для користувача {user_id}:")
for pred in top_predictions:
    print(f"Фільм ID: {pred.iid}, Прогнозована оцінка: {pred.est:.2f}")

# Збереження моделі
dump('best_model.pkl', algo=best_algo)

Використання готової бібліотеки Surprise дозволяє швидко експериментувати з різними алгоритмами та легко оптимізувати їх параметри. З іншого боку, створення власної системи дає більше контролю над процесом та допомагає краще зрозуміти принципи роботи рекомендаційних систем.